# Exploratory Data Analysis
---

**Objective:** Understand the variables, association between them and necessary transformations before model selection/building.

**Process:**
1. Importing libraries and dataset
2. Checking data types, nulls and outliers
3. Univariate analysis (target)
4. Bivariate analysis (relation to target)
5. Multivariate analysis (multicollinearity)
---

## 1. Importing libraries and dataset

In [6]:
# Adding src to the path for modules
import sys
import os
root_path = os.path.abspath('..')
src_path = os.path.join(root_path, 'src')
sys.path.append(src_path)

# Data Manipulation
import pandas as pd
import numpy as np

# Hypothesis Test
import scipy.stats as stats

# Data Viz
import matplotlib.pyplot as plt
import seaborn as sns

# Project variables and functions
import config 
import utils 

Importing the data and having a first look at it:

In [2]:
# Importing
df = utils.read_compressed_data(config.COLLECTED_DATA_PATH)

# First look
df.sample(10)

,year,quarter,state,age,sex,race,literate,highest_educational_level,years_studied,worker_type,work_segment,occupation_group,tax_payer,weekly_worked_hours,main_work_income,all_work_income,weekly_worked_hours_all_jobs
1499386,2022,3,SC,16,2,1,1,4,10,1,9,2,1,2,1000.0,1000.0,30
1156802,2022,1,GO,50,1,1,1,5,12,9,7,3,2,4,7000.0,7000.0,48
1249078,2023,3,RJ,36,2,1,1,7,16,7,9,2,1,2,7000.0,7000.0,25
262697,2023,3,RJ,42,2,1,1,6,13,8,10,3,2,3,5000.0,5000.0,44
906794,2021,4,CE,45,2,4,2,2,2,4,11,9,2,1,30.0,30.0,4
753409,2021,3,SP,34,2,4,1,5,12,4,11,9,2,2,960.0,960.0,20
1938662,2021,3,MG,27,2,4,1,5,12,2,2,7,2,2,1100.0,1100.0,20
1381581,2022,1,SP,26,2,4,1,3,9,1,6,9,1,3,1300.0,1300.0,44
668183,2021,4,MA,22,2,4,1,4,11,9,1,9,2,5,300.0,700.0,60
127752,2023,4,RN,54,1,4,2,1,0,1,3,3,1,4,3000.0,3000.0,48


---
## 2. Checking data types, nulls and outliers

Checking types and nulls:

In [9]:
df.isnull().mean().round(2) * 100

year                            0.0
quarter                         0.0
state                           0.0
age                             0.0
sex                             0.0
race                            0.0
literate                        0.0
highest_educational_level       0.0
years_studied                   0.0
worker_type                     0.0
work_segment                    0.0
occupation_group                0.0
tax_payer                       0.0
weekly_worked_hours             0.0
main_work_income                0.0
all_work_income                 0.0
weekly_worked_hours_all_jobs    0.0
dtype: float64